In [1]:
import sys
from pathlib import Path
from threading import Thread
from subprocess import Popen, PIPE
from collections import defaultdict

import numpy as np
import pandas as pd

In [2]:
!ls ../out

2022-08-14T22:24:50.595949_6397.73_sa_5_0.5
2022-08-14T22:27:21.957662_6670.33_sa_double_tl
2022-08-14T22:41:23.433876_7152.92_sa_5x_tl


In [3]:
scores = {}

def read_stream(name, in_file, out_file):
    for line in in_file:
        #print(f"[{name}] {line.strip()}", file=out_file)
        try:
            scores[name] = int(line.strip().split()[-1])
        except:
            pass

def run(cmd, name):
    proc = Popen(cmd, stdout=PIPE, stderr=PIPE, universal_newlines=True, shell=True)
    stdout_thread = Thread(target=read_stream, args=(name, proc.stdout, sys.stdout))
    stderr_thread = Thread(target=read_stream, args=(name, proc.stderr, sys.stderr))
    stdout_thread.start()
    stderr_thread.start()
    proc.wait()
    return proc

N = 100
for out_dir in Path("../out").iterdir():
    for i in range(N):
        out_file = out_dir / f"{i:04d}.txt"
        cmd = f"../tools/target/release/vis ../tools/in/{i:04d}.txt {out_file}"
        run(cmd, (out_dir, i))

In [4]:
dict_out_dir_to_scores = defaultdict(lambda: [0] * N)
for (out_dir, i), score in scores.items():
    dict_out_dir_to_scores[out_dir][i] = score

In [5]:
df = pd.DataFrame(dict_out_dir_to_scores)
df.corr()

,../out/2022-08-14T22:27:21.957662_6670.33_sa_double_tl,../out/2022-08-14T22:41:23.433876_7152.92_sa_5x_tl,../out/2022-08-14T22:24:50.595949_6397.73_sa_5_0.5
../out/2022-08-14T22:27:21.957662_6670.33_sa_double_tl,1.000000,0.874795,0.929135
../out/2022-08-14T22:41:23.433876_7152.92_sa_5x_tl,0.874795,1.000000,0.873963
../out/2022-08-14T22:24:50.595949_6397.73_sa_5_0.5,0.929135,0.873963,1.000000


In [6]:
df.values.argmax(1)

array([1, 1, 1, 0, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0,
       1, 1, 1, 2, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1])

In [7]:
np.bincount(df.values.argmax(1))

array([25, 71,  4])

In [8]:
df.columns

Index([../out/2022-08-14T22:27:21.957662_6670.33_sa_double_tl,
           ../out/2022-08-14T22:41:23.433876_7152.92_sa_5x_tl,
           ../out/2022-08-14T22:24:50.595949_6397.73_sa_5_0.5],
      dtype='object')